In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Price opt.csv to Price opt.csv


In [ ]:
df = pd.read_csv('Price opt.csv')

In [ ]:
df.head()

,product_id,product_name,our_price,walmart_price,ebay_price,customer_id,location,clicks,cart_additions,purchased,inventory_level,date,customer_segment,adjusted_price,revenue_generated
0,P007,Fitbit Charge 5,81.69,78.52,84.49,C6734,FL,4,1,1,87,2025-04-12,Buyer,81.69,81.69
1,P006,Amazon Echo Dot,10.07,14.99,11.24,C4385,NY,4,1,1,189,2025-04-30,Buyer,9.87,9.87
2,P015,Anker PowerCore 10000,65.65,64.47,70.48,C2528,FL,2,0,0,131,2025-04-25,Clicker,65.65,0.00
3,P014,JBL Flip 6,82.76,80.81,78.74,C7235,FL,3,2,1,103,2025-04-04,Buyer,82.76,82.76
4,P002,Samsung Galaxy S21,48.26,45.34,48.94,C7873,CA,10,3,0,187,2025-04-28,Cart Abandoner,48.74,0.00


In [ ]:
# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.head()

,product_id,product_name,our_price,walmart_price,ebay_price,customer_id,location,clicks,cart_additions,purchased,inventory_level,date,customer_segment,adjusted_price,revenue_generated
0,P007,Fitbit Charge 5,81.69,78.52,84.49,C6734,FL,4,1,1,87,2025-04-12,Buyer,81.69,81.69
1,P006,Amazon Echo Dot,10.07,14.99,11.24,C4385,NY,4,1,1,189,2025-04-30,Buyer,9.87,9.87
2,P015,Anker PowerCore 10000,65.65,64.47,70.48,C2528,FL,2,0,0,131,2025-04-25,Clicker,65.65,0.00
3,P014,JBL Flip 6,82.76,80.81,78.74,C7235,FL,3,2,1,103,2025-04-04,Buyer,82.76,82.76
4,P002,Samsung Galaxy S21,48.26,45.34,48.94,C7873,CA,10,3,0,187,2025-04-28,Cart Abandoner,48.74,0.00


In [ ]:
# Check for duplicates and remove them
df = df.drop_duplicates()

# Check for missing values
missing = df.isnull().sum()
print("Missing Values:\n", missing)

# Data type conversions
num_cols = ['our_price', 'walmart_price', 'ebay_price', 'clicks',
            'cart_additions', 'purchased', 'inventory_level',
            'adjusted_price', 'revenue_generated']
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')

# Standardize column names (optional)
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')


Missing Values:
 product_id           0
product_name         0
our_price            0
walmart_price        0
ebay_price           0
customer_id          0
location             0
clicks               0
cart_additions       0
purchased            0
inventory_level      0
date                 0
customer_segment     0
adjusted_price       0
revenue_generated    0
dtype: int64


In [ ]:
# feature engineering
# Price difference from competitors
df['price_vs_walmart'] = df['our_price'] - df['walmart_price']
df['price_vs_ebay'] = df['our_price'] - df['ebay_price']



In [ ]:
# Conversion rate (if clicks > 0)
df['conversion_rate'] = df.apply(lambda row: row['purchased'] / row['clicks']
                                 if row['clicks'] > 0 else 0, axis=1)

# Engagement score (simple weighting: clicks + 2×cart_additions + 3×purchased)
df['engagement_score'] = df['clicks'] + 2 * df['cart_additions'] + 3 * df['purchased']

# Stock-out risk flag
df['stock_risk'] = df['inventory_level'].apply(lambda x: 1 if x < 20 else 0)

In [ ]:
df.head()

,product_id,product_name,our_price,walmart_price,ebay_price,customer_id,location,clicks,cart_additions,purchased,inventory_level,date,customer_segment,adjusted_price,revenue_generated,price_vs_walmart,price_vs_ebay,conversion_rate,engagement_score,stock_risk
0,P007,Fitbit Charge 5,81.69,78.52,84.49,C6734,FL,4,1,1,87,2025-04-12,Buyer,81.69,81.69,3.17,-2.80,0.250000,9,0
1,P006,Amazon Echo Dot,10.07,14.99,11.24,C4385,NY,4,1,1,189,2025-04-30,Buyer,9.87,9.87,-4.92,-1.17,0.250000,9,0
2,P015,Anker PowerCore 10000,65.65,64.47,70.48,C2528,FL,2,0,0,131,2025-04-25,Clicker,65.65,0.00,1.18,-4.83,0.000000,2,0
3,P014,JBL Flip 6,82.76,80.81,78.74,C7235,FL,3,2,1,103,2025-04-04,Buyer,82.76,82.76,1.95,4.02,0.333333,10,0
4,P002,Samsung Galaxy S21,48.26,45.34,48.94,C7873,CA,10,3,0,187,2025-04-28,Cart Abandoner,48.74,0.00,2.92,-0.68,0.000000,16,0


In [ ]:
# Data quality metrics
def assess_data_quality(data):
    print("Data Quality Assessment:")
    print("-" * 30)
    print("Accuracy: Check if adjusted price equals our price where purchased = 1")
    mismatches = data[(data['purchased'] == 1) & (data['adjusted_price'] != data['our_price'])]
    print(f"Inconsistent adjusted prices: {len(mismatches)} rows")

    print("\nCompleteness:")
    print(data.isnull().sum())

    print("\nConsistency: Unique product names per product_id")
    print(data.groupby('product_id')['product_name'].nunique().value_counts())

    print("\nValidity: Any negative prices?")
    print((data[['our_price', 'walmart_price', 'ebay_price']] < 0).sum())

    print("\nTimeliness: Date range")
    print(f"From {data['date'].min()} to {data['date'].max()}")

assess_data_quality(df)


Data Quality Assessment:
------------------------------
Accuracy: Check if adjusted price equals our price where purchased = 1
Inconsistent adjusted prices: 164 rows

Completeness:
product_id           0
product_name         0
our_price            0
walmart_price        0
ebay_price           0
customer_id          0
location             0
clicks               0
cart_additions       0
purchased            0
inventory_level      0
date                 0
customer_segment     0
adjusted_price       0
revenue_generated    0
price_vs_walmart     0
price_vs_ebay        0
conversion_rate      0
engagement_score     0
stock_risk           0
dtype: int64

Consistency: Unique product names per product_id
product_name
1    20
Name: count, dtype: int64

Validity: Any negative prices?
our_price        0
walmart_price    0
ebay_price       0
dtype: int64

Timeliness: Date range
From 2025-04-01 00:00:00 to 2025-04-30 00:00:00
